In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

walmart_recruiting_store_sales_forecasting_path = kagglehub.competition_download('walmart-recruiting-store-sales-forecasting')

print('Data source import complete.')


100%|██████████| 2.70M/2.70M [00:00<00:00, 156MB/s]

Extracting files...
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ML/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 640MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import holidays
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
import gc
import joblib
import os
from datetime import datetime
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:

stores = pd.read_csv('stores.csv')
train = pd.read_csv("train.csv.zip")
features = pd.read_csv('features.csv.zip')
sample = pd.read_csv('sampleSubmission.csv.zip')
test = pd.read_csv('test.csv.zip')

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
features['Date'] = pd.to_datetime(features['Date'])
train_df = pd.merge(train, features, on=['Store', 'Date', 'IsHoliday'], how='left')
test_df = pd.merge(test, features, on=['Store', 'Date', 'IsHoliday'], how='left')
train_df = pd.merge(train_df, stores, on='Store', how='left')
test_df = pd.merge(test_df, stores, on='Store', how='left')
print("\n--- Merged Train Data Info ---")
print(train_df.info())
print("\n--- Merged Test Data Info ---")
print(test_df.info())
X_train = train_df.drop(['Weekly_Sales'], axis=1)
y_train = train_df['Weekly_Sales']

del train, test, features, stores
gc.collect()



--- Merged Train Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     150681 non-null  float64       
 8   MarkDown2     111248 non-null  float64       
 9   MarkDown3     137091 non-null  float64       
 10  MarkDown4     134967 non-null  float64       
 11  MarkDown5     151432 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  objec

8

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MissingMarkdownHandler:

    def __init__(self):
        self.markdown_cols = None

    def fit(self, X, y=None):
        self.markdown_cols = [col for col in X.columns if 'markdown' in col.lower()]
        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.markdown_cols:
            if col in X_copy.columns:
                X_copy[f'{col}_was_missing'] = X_copy[col].isna().astype(int)

                X_copy[col] = X_copy[col].fillna(0)

        return X_copy

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)



In [ ]:
class MissingValueImputer(BaseEstimator, TransformerMixin):

    def __init__(self, numerical_cols_to_impute=None):
        self.numerical_cols_to_impute = numerical_cols_to_impute if numerical_cols_to_impute is not None else ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
        self.means = {}

    def fit(self, X, y=None):
        for col in self.numerical_cols_to_impute:
            if col in X.columns:
                self.means[col] = X[col].mean()
        return self

    def transform(self, X):
        X_copy = X.copy()

        for col in self.numerical_cols_to_impute:
            if col in X_copy.columns:
                X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')
                if X_copy[col].isnull().any() and col in self.means:
                    X_copy[col] = X_copy[col].fillna(self.means[col])
        return X_copy


In [ ]:
class AdvancedDateFeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, date_column: str = 'Date', include_holidays: bool = True,
                 include_seasonal: bool = True, include_lags: bool = False):
        self.date_column = date_column
        self.include_holidays = include_holidays
        self.include_seasonal = include_seasonal
        self.include_lags = include_lags

    def _is_holiday_period(self, date):
        month, day = date.month, date.day

        if month == 11 and day >= 22:
            return 1
        elif month == 12:
            return 1
        elif month == 1 and day <= 7:
            return 1
        elif month == 9 and day <= 7:
            return 1
        elif month == 5 and day >= 25:
            return 1
        else:
            return 0

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()

        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek
        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Quarter'] = X_copy[self.date_column].dt.quarter
        X_copy['DayOfYear'] = X_copy[self.date_column].dt.dayofyear

        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)
        X_copy['DayOfWeek_sin'] = np.sin(2 * np.pi * X_copy['DayOfWeek'] / 7)
        X_copy['DayOfWeek_cos'] = np.cos(2 * np.pi * X_copy['DayOfWeek'] / 7)

        if self.include_seasonal:
            X_copy['Season'] = X_copy['Month'].map({12: 0, 1: 0, 2: 0,
                                                   3: 1, 4: 1, 5: 1,
                                                   6: 2, 7: 2, 8: 2,
                                                   9: 3, 10: 3, 11: 3})

            X_copy['IsWeekend'] = (X_copy['DayOfWeek'] >= 5).astype(int)

            X_copy['IsMonthEnd'] = (X_copy[self.date_column].dt.is_month_end).astype(int)
            X_copy['IsMonthStart'] = (X_copy[self.date_column].dt.is_month_start).astype(int)

        if self.include_holidays:
            X_copy['IsHolidayPeriod'] = X_copy[self.date_column].apply(self._is_holiday_period)

            if 'IsHoliday' in X_copy.columns:
                if X_copy['IsHoliday'].dtype == bool:
                    X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        markdown_cols = [col for col in X_copy.columns if col.startswith('MarkDown') and not col.endswith('_was_missing')]
        if markdown_cols:
            X_copy['Total_MarkDown'] = X_copy[markdown_cols].sum(axis=1)
            X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1e-8)
            X_copy['HasMarkDown'] = (X_copy['Total_MarkDown'] > 0).astype(int)

        econ_cols = ['Fuel_Price', 'CPI', 'Unemployment']
        available_econ = [col for col in econ_cols if col in X_copy.columns]

        if len(available_econ) >= 2:
            if 'Fuel_Price' in X_copy.columns and 'CPI' in X_copy.columns:
                X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / (X_copy['CPI'] + 1e-8)

            if 'CPI' in X_copy.columns and 'Unemployment' in X_copy.columns:
                X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100

        return X_copy


In [ ]:
class DateFeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, date_column='Date'):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.date_column not in X_copy.columns:
            raise ValueError(f"Date column '{self.date_column}' not found in DataFrame.")

        X_copy[self.date_column] = pd.to_datetime(X_copy[self.date_column])

        X_copy['Year'] = X_copy[self.date_column].dt.year
        X_copy['Month'] = X_copy[self.date_column].dt.month
        X_copy['Month_sin'] = np.sin(2 * np.pi * X_copy['Month'] / 12)
        X_copy['Month_cos'] = np.cos(2 * np.pi * X_copy['Month'] / 12)

        X_copy['Week'] = X_copy[self.date_column].dt.isocalendar().week.astype(int)
        X_copy['Day'] = X_copy[self.date_column].dt.day
        X_copy['DayOfWeek'] = X_copy[self.date_column].dt.dayofweek

        X_copy['Week_sin'] = np.sin(2 * np.pi * X_copy['Week'] / 52)
        X_copy['Week_cos'] = np.cos(2 * np.pi * X_copy['Week'] / 52)

        X_copy['Total_MarkDown'] = X_copy[[f'MarkDown{i}' for i in range(1, 6)]].sum(axis=1)
        X_copy['MarkDown_Intensity'] = X_copy['Total_MarkDown'] / (X_copy['Total_MarkDown'].mean() + 1)

        X_copy['Fuel_CPI_Ratio'] = X_copy['Fuel_Price'] / X_copy['CPI']
        X_copy['Economic_Index'] = (X_copy['CPI'] * 0.4 + (100 - X_copy['Unemployment']) * 0.6) / 100


        if 'IsHoliday' in X_copy.columns and X_copy['IsHoliday'].dtype == bool:
            X_copy['IsHoliday'] = X_copy['IsHoliday'].astype(int)

        return X_copy

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

preprocessing_pipeline = Pipeline([
    ('missing_markdown', MissingMarkdownHandler()),
    ('missing_imputer', MissingValueImputer()),
    ('date_features', AdvancedDateFeatureExtractor())
])

In [ ]:
X_train_processed = preprocessing_pipeline.fit_transform(X_train, y_train)


/tmp/ipython-input-1507361449.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')


In [ ]:
if 'Date' in X_train_processed.columns:
    dates = X_train_processed['Date']
    X_train_processed = X_train_processed.drop(['Date'], axis=1)

print(f"Training data shape: {X_train_processed.shape}")
print(f"Features: {list(X_train_processed.columns)}")

print("\n--- Applying Preprocessing Pipeline to Test Data ---")
X_test_processed = preprocessing_pipeline.transform(test_df.drop(columns=['Id'], errors='ignore'))

if 'Date' in X_test_processed.columns:
    dates = X_test_processed['Date']
    X_test_processed = X_test_processed.drop(['Date'], axis=1)

print("\nProcessed X_train_processed info:")
print(X_train_processed.info())
print("\nProcessed X_test_processed info:")
print(X_test_processed.info())

print("\nMissing values in processed X_train_processed:\n", X_train_processed.isnull().sum().sum())
print("Missing values in processed X_test_processed:\n", X_test_processed.isnull().sum().sum())

Training data shape: (421570, 42)
Features: ['Store', 'Dept', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'MarkDown1_was_missing', 'MarkDown2_was_missing', 'MarkDown3_was_missing', 'MarkDown4_was_missing', 'MarkDown5_was_missing', 'Year', 'Month', 'Day', 'DayOfWeek', 'Week', 'Quarter', 'DayOfYear', 'Month_sin', 'Month_cos', 'Week_sin', 'Week_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'Season', 'IsWeekend', 'IsMonthEnd', 'IsMonthStart', 'IsHolidayPeriod', 'Total_MarkDown', 'MarkDown_Intensity', 'HasMarkDown', 'Fuel_CPI_Ratio', 'Economic_Index']

--- Applying Preprocessing Pipeline to Test Data ---


/tmp/ipython-input-1507361449.py:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_copy[col] = X_copy[col].fillna(method='ffill').fillna(method='bfill')



Processed X_train_processed info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Store                  421570 non-null  int64  
 1   Dept                   421570 non-null  int64  
 2   IsHoliday              421570 non-null  int64  
 3   Temperature            421570 non-null  float64
 4   Fuel_Price             421570 non-null  float64
 5   MarkDown1              421570 non-null  float64
 6   MarkDown2              421570 non-null  float64
 7   MarkDown3              421570 non-null  float64
 8   MarkDown4              421570 non-null  float64
 9   MarkDown5              421570 non-null  float64
 10  CPI                    421570 non-null  float64
 11  Unemployment           421570 non-null  float64
 12  Type                   421570 non-null  object 
 13  Size                   421570 non-null  int64  
 14  M

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
features_after_pipeline = X_train_processed.columns.tolist()
categorical_features_after_pipeline = [col for col in ['Store', 'Dept', 'Type'] if col in features_after_pipeline]


In [ ]:

train_weights = np.where(X_train_processed['IsHoliday'] == 1, 5, 1)

test_ids = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date'].dt.strftime('%Y-%m-%d')

temp_train_df = X_train_processed.copy()
temp_train_df['Date'] = pd.to_datetime(train_df['Date'])

temp_train_df['Weekly_Sales'] = y_train
temp_train_df = temp_train_df.sort_values(by='Date').reset_index(drop=True)

validation_cutoff_date = pd.to_datetime('2012-07-01')

features_for_training = [col for col in features_after_pipeline if col != 'Date']

X_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date][features_for_training]
y_train_split = temp_train_df[temp_train_df['Date'] < validation_cutoff_date]['Weekly_Sales']
X_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date][features_for_training]
y_val_split = temp_train_df[temp_train_df['Date'] >= validation_cutoff_date]['Weekly_Sales']

def weighted_mean_absolute_error(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

val_weights = np.where(X_val_split['IsHoliday'] == 1, 5, 1)
train_weights_split = np.where(X_train_split['IsHoliday'] == 1, 5, 1)

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import LabelEncoder

lgb_params = {
    'objective': 'regression_l1',
    'metric': 'mae',
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
    'max_depth': -1,
}

param_grid = {
    'num_leaves': [32, 64, 128]
}


best_score = float('inf')
best_params = None

le = LabelEncoder()
X_train_split['Type'] = le.fit_transform(X_train_split['Type'])
X_val_split['Type'] = le.transform(X_val_split['Type'])
X_test_processed['Type'] = le.transform(X_test_processed['Type'])


for params in ParameterGrid(param_grid):
    model = lgb.LGBMRegressor(**params, **lgb_params)
    model.fit(X_train_split, y_train_split,
              eval_set=[(X_val_split, y_val_split)],
              callbacks=[lgb.early_stopping(50)],
              categorical_feature=['Store', 'Dept', 'Type'])

    val_preds = model.predict(X_val_split)
    train_weights_split = np.where(X_train_split['IsHoliday'] == 1, 5, 1)
    val_wmae = weighted_mean_absolute_error(y_val_split, val_preds, val_weights)
    print(f"WMAE on val set: {val_wmae:.4f}")

    train_predictions = model.predict(X_train_split)
    train_wmae = weighted_mean_absolute_error(y_train_split, train_predictions, train_weights_split)
    print(f"WMAE on training set: {train_wmae:.4f}")

    print("--------------------------------------------------")
    if val_wmae < best_score:
        best_score = val_wmae
        best_params = params
        best_model = model

print("Best WMAE:", best_score)
print("Best hyperparameters:", best_params)

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's l1: 2034.72
WMAE on val set: 2101.5078
WMAE on training set: 2347.4416
--------------------------------------------------
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's l1: 1755.68
WMAE on val set: 1814.5404
WMAE on training set: 1931.3508
--------------------------------------------------
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[2000]	valid_0's l1: 1599.37
WMAE on val set: 1649.9871
WMAE on training set: 1632.7118
--------------------------------------------------
Best WMAE: 1649.9871043973637
Best hyperparameters: {'num_leaves': 128}


In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.7 MB/s eta 0:00:00


In [ ]:
import mlflow
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"
mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

wmae = 1649.9871043973637

with mlflow.start_run(run_name = "LightGBM_best"):
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("n_estimators", 2000)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("max_depth", -1)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("num_leaves", 128)

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")

wmae = 1814.5404

with mlflow.start_run(run_name = "LightGBM_64"):
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("n_estimators", 2000)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("max_depth", -1)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("num_leaves", 64)

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")

wmae = 2034.72

with mlflow.start_run(run_name = "LightGBM_32"):
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("n_estimators", 2000)
    mlflow.log_param("learning_rate", 0.02)
    mlflow.log_param("max_depth", -1)
    mlflow.log_param("subsample", 0.8)
    mlflow.log_param("num_leaves", 32)

    mlflow.log_metric("final_wmae", wmae)

    print("Logged successfully!")







Logged successfully!
🏃 View run LightGBM_best at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/2de189413b2449f69f47ca1626b88764
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
Logged successfully!
🏃 View run LightGBM_64 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/71bc59640630420084b51c74819ee215
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
Logged successfully!
🏃 View run LightGBM_32 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/b8543b3331574ca0947d9741bb30b29b
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
